In [4]:
# Step 1: Import Required Libraries
print("=== STEP 1: IMPORTING LIBRARIES ===")

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import joblib
import pickle
import json
import os
from datetime import datetime

# Model training and evaluation
from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.metrics import (accuracy_score, precision_score, recall_score,
                           f1_score, roc_auc_score, confusion_matrix,
                           classification_report, roc_curve, precision_recall_curve)
from sklearn.preprocessing import StandardScaler

# Hyperparameter optimization
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
import warnings
warnings.filterwarnings('ignore')

# Configuration
plt.style.use('seaborn-v0_8')
np.random.seed(42)

print("✅ All libraries imported successfully!")

=== STEP 1: IMPORTING LIBRARIES ===
✅ All libraries imported successfully!


In [5]:
# Step 2: Load Data and Available Models
print("=== STEP 2: LOADING DATA AND AVAILABLE MODELS ===")

# Load processed data
df = pd.read_csv('../data/processed/telco_processed.csv')
print(f"Dataset shape: {df.shape}")

# Handle categorical variables if any
object_cols = df.select_dtypes(include=['object']).columns
if len(object_cols) > 0:
    print(f"Categorical columns found: {list(object_cols)}")
    for col in object_cols:
        df[col] = pd.factorize(df[col])[0]

# Prepare features and target
X = df.drop('Churn', axis=1)
y = df['Churn']
feature_names = X.columns.tolist()

print(f"Features: {len(feature_names)}")
print(f"Target distribution: {y.value_counts().to_dict()}")

# Load all available models
def load_all_models():
    models = {}
    model_files = {
        'RandomForest': '../models/randomforest_model.pkl',
        'XGBoost': '../models/xgboost_model.pkl',
        'LightGBM': '../models/lightgbm_model.pkl',
        'AdaBoost': '../models/adaboost_model.pkl'
    }

    for name, path in model_files.items():
        try:
            models[name] = joblib.load(path)
            print(f"✅ {name} loaded successfully")
        except Exception as e:
            print(f"❌ {name} not available: {e}")

    return models

available_models = load_all_models()
print(f"\n🎯 Available models: {list(available_models.keys())}")

=== STEP 2: LOADING DATA AND AVAILABLE MODELS ===
Dataset shape: (7009, 16)
Features: 15
Target distribution: {0: 5152, 1: 1857}
✅ RandomForest loaded successfully
✅ XGBoost loaded successfully
✅ LightGBM loaded successfully
✅ AdaBoost loaded successfully

🎯 Available models: ['RandomForest', 'XGBoost', 'LightGBM', 'AdaBoost']


In [ ]:
# Step 3: Determine Best Performing Model
print("=== STEP 3: DETERMINING BEST PERFORMING MODEL ===")

def evaluate_models_on_full_data(models, X, y):
    """Evaluate all models using cross-validation on full dataset"""

    cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    evaluation_results = {}

    for name, model in models.items():
        print(f"Evaluating {name}...")

        try:
            # Cross-validation scores
            cv_accuracy = cross_val_score(model, X, y, cv=cv, scoring='accuracy')
            cv_auc = cross_val_score(model, X, y, cv=cv, scoring='roc_auc')

            evaluation_results[name] = {
                'model': model,
                'cv_accuracy_mean': cv_accuracy.mean(),
                'cv_accuracy_std': cv_accuracy.std(),
                'cv_auc_mean': cv_auc.mean(),
                'cv_auc_std': cv_auc.std(),
                'cv_score': (cv_accuracy.mean() + cv_auc.mean()) / 2  # Combined score
            }

            print(f"  {name}: CV Accuracy = {cv_accuracy.mean():.4f} (±{cv_accuracy.std():.4f}), "
                  f"AUC = {cv_auc.mean():.4f} (±{cv_auc.std():.4f})")

        except Exception as e:
            print(f"  ❌ Evaluation failed for {name}: {e}")

    return evaluation_results

# Evaluate all models
evaluation_results = evaluate_models_on_full_data(available_models, X, y)

# Determine best model
if evaluation_results:
    best_model_name = max(evaluation_results.keys(),
                         key=lambda x: evaluation_results[x]['cv_score'])
    best_model_info = evaluation_results[best_model_name]

    print(f"\n🏆 BEST MODEL: {best_model_name}")
    print(f"   CV Accuracy: {best_model_info['cv_accuracy_mean']:.4f} (±{best_model_info['cv_accuracy_std']:.4f})")
    print(f"   CV AUC: {best_model_info['cv_auc_mean']:.4f} (±{best_model_info['cv_auc_std']:.4f})")
    print(f"   Combined Score: {best_model_info['cv_score']:.4f}")
else:
    print("❌ No models were successfully evaluated")
    best_model_name = "RandomForest"  # Fallback
    best_model_info = None

=== STEP 3: DETERMINING BEST PERFORMING MODEL ===
Evaluating RandomForest...
  RandomForest: CV Accuracy = 0.7941 (±0.0062), AUC = 0.8427 (±0.0105)
Evaluating XGBoost...
  XGBoost: CV Accuracy = 0.7993 (±0.0098), AUC = 0.8448 (±0.0110)
Evaluating LightGBM...
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Number of positive: 1486, number of negative: 4121
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.002014 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1107
[LightGBM] [Info] Number of data points in the train set: 5607, number of used features: 15
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.265026 -> initscore=-1.020008
[LightGBM] [Info] Start training from score -1.020008
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further